<p><img alt="FruitSalad logo" height="auto" width="45" src="https://github.com/ctinnil/FruitSalad/blob/main/res/favicon.png?raw=true" align="left" hspace="10px" vspace="0px"></p>
<h1>Developing the training dataset for the FruitSalad system</h1>

FruitSalad represents our POC for a full reactive WiFi honeypot. Try it out, express your opinion and experience to improve the framework and minimize the attack surface for a better, safer, quicker, and opener Internet.

Dubbed FruitSalad due to its versatility and ability to be configured on popular small single-board computers as [Raspberry Pi](https://www.raspberrypi.org/), [Orange Pi](http://www.orangepi.org/), [Banana Pi](http://www.banana-pi.org) or [NanoPi](http://nanopi.io/nanopi-m3.html), this framework offers a new approach to WiFi networks security.

Intrusion detection will not suffice in today's threat landscape. By taking a more proactive approach to attackers, a reactive WiFi honeypot can deter further aggressors.

**This first notebook focuses on the development of a custom dataset for our problem. We begin by monitoring for different attacks and labeling the observed TTPs. ** 

In [ ]:
# Check system configuration (OS, Python version)

!python3 --version
!uname -a
!lsb_release -a | grep Release

Python 3.9.2
Linux kali 5.10.0-kali7-amd64 #1 SMP Debian 5.10.28-1kali1 (2021-04-12) x86_64 GNU/Linux
Release:        2021.2


In [ ]:
# Install dependencies

!pip install keras
!pip install sklearn
!pip install scapy

## Before you begin, you need to

### set capabilities for binaries that need elevated permissions (root)

```zsh
sudo setcap cap_net_raw=eip /usr/bin/python3.9 
sudo setcap cap_net_raw=eip /usr/bin/scapy
sudo setcap cap_net_raw=eip /usr/bin/tcpdump
```
### set your WiFi interface in monitoring mode

#### you can either use airmon-ng or the default network managing command-line tools

```zsh
sudo airmon-ng start wlan0

# OR 

sudo ifconfig wlan0 down
sudo iwconfig wlan0 mode monitor
sudo ifconfig wlan0 up

# OR

sudo ip link set wlan0 down
sudo iw wlan0 set monitor none
sudo ip link set wlan0 up
```

#### to switch back 

```zsh
sudo airmon-ng stop wlan0mon

# OR

sudo ifconfig wlan0 down
sudo iwconfig wlan0 mode managed
sudo ifconfig wlan0 up

# OR

sudo ip link set wlan0 down
sudo iw wlan0 set type managed
sudo ip link set wlan0 up
```

## Experimentation network

<p align="center">
  <img src="https://github.com/ctinnil/FruitSalad/blob/main/res/net.png?raw=true" />
</p>

To simulate multiple devices, you can change the physical addresses of specific devices. Moreover, try to have all types of devices (workstation, mobile, and IoT).

## In this version of the dataset, we proposed the following labels, based on attacker experience and actions 

1. "skiddie" = "script_kiddie"
2. "hacker" = "motivated_attacker"
3. "HEO" = "human_error_or_normal_forget"

In [5]:
# Import dependencies

from datetime import datetime
from sklearn import preprocessing
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense
from numpy import loadtxt
import pandas as pd
import math

from scapy.all import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Listen for different attacks and record Deauthentication and Dissociation to dataframe

df = pd.DataFrame(columns=['time', 'client_addr', 'AP_addr', 'BSSID', 'addr4', 'version', 'pad', 'present', 'Rate', 'len', 'TXFlags', 'notdecoded', 'proto', 'subtype', 'type', 'cfe', 'FCfield', 'ID', 'SC', 'Flags', 'ChannelFrequency', 'dBm_AntSignal', 'ChannelFlags', 'reason', 'label', 'obs'])
pkts = sniff(iface='wlan0', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=0
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'skiddie'+','+'sudo aireplay-ng -0 0 -a 98:de:d0:XX:XX:XX wlan0 '
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA

In [ ]:
# Checking dataframe 

df.head()

,time,client_addr,AP_addr,BSSID,addr4,version,pad,present,Rate,len,...,FCfield,ID,SC,Flags,ChannelFrequency,dBm_AntSignal,ChannelFlags,reason,label,obs
0,[2021-06-02 14:25:09],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS,1.0,12,...,,14849,0,NONE,NONE,NONE,NONE,7,skiddie,sudo aireplay-ng -0 0 -a 98:de:d0:XX:XX:XX wlan0
1,[2021-06-02 14:25:09],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS,1.0,12,...,,14849,16,NONE,NONE,NONE,NONE,7,skiddie,sudo aireplay-ng -0 0 -a 98:de:d0:XX:XX:XX wlan0
2,[2021-06-02 14:25:09],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS+B18,1.0,13,...,,14849,0,NONE,NONE,NONE,NONE,7,skiddie,sudo aireplay-ng -0 0 -a 98:de:d0:XX:XX:XX wlan0
3,[2021-06-02 14:25:09],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS,1.0,12,...,,14849,32,NONE,NONE,NONE,NONE,7,skiddie,sudo aireplay-ng -0 0 -a 98:de:d0:XX:XX:XX wlan0
4,[2021-06-02 14:25:09],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS+B18,1.0,13,...,,14849,16,NONE,NONE,NONE,NONE,7,skiddie,sudo aireplay-ng -0 0 -a 98:de:d0:XX:XX:XX wlan0


In [ ]:
pkts = sniff(iface='wlan0', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=100
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'skiddie'+','+'sudo wifite -c11'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA

In [ ]:
df.tail(1)

,time,client_addr,AP_addr,BSSID,addr4,version,pad,present,Rate,len,...,FCfield,ID,SC,Flags,ChannelFrequency,dBm_AntSignal,ChannelFlags,reason,label,obs
199,[2021-06-03 00:00:25],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS+B18,1.0,13,...,,14849,768,NONE,NONE,NONE,NONE,7,skiddie,sudo wifite -c11


In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=200
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'skiddie'+','+'sudo besside-ng -W -v wlan0mon'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > ff:ff:ff:ff:ff:ff (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=300
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'skiddie'+','+'sudo fern-wifi-cracker'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 24:62:ab:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 24:62:ab:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 24:62:ab:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 24:62:ab:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 24:62:ab:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 24:62:ab:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 24:62:ab:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 24:62:ab:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=400
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'hacker'+','+'sudo aireplay-ng -010 -a 98:de:d0:XX:XX:XX  -c 24:62:ab:XX:XX:XX wlan0mon & sudo aireplay-ng -010 -a 98:de:d0:XX:XX:XX  -c 46:fb:37:XX:XX:XX wlan0mon & '
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 24:62:ab:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 24:62:ab:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 24:62:ab:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 24:62:ab:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=500
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'hacker'+','+'sudo ./airgeddon.sh'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:X

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=600
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'hacker'+','+'sudo ./airgeddon.sh #DDoS'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Disassociation 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:X

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=700
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'hacker'+','+'sudo python3 deauthenticator.py -i wlan0mon -c5 -a 98:de:d0:XX:XX:XX -t 46:fb:37:XX:XX:XX # scapy version'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=800
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'hacker'+','+'# Special hardware # WiFi Pineapple Nano #'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA

In [ ]:
pkts = sniff(iface='wlan0mon', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=900
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'HEO'+','+'other'
  df.loc[i] = list(a.split(','))
  i=i+1

RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 46:fb:37:XX:XX:XX (TA=SA

In [ ]:
pkts = sniff(iface='wlan0', count=100, filter='type mgt and (subtype deauth or subtype disassoc)' ,prn=lambda x:x.summary())
i=1000
for pkt in pkts:
  timestamp = datetime.fromtimestamp(pkt.time)
  #print(timestamp)
  a='['+timestamp.strftime('%Y-%m-%d %H:%M:%S')+']'+','+str(pkt.addr1).upper()+','+str(pkt.addr2).upper()+','+ \
        str(pkt.addr3).upper()+','+str(pkt.addr4).upper()+','+str(pkt.version).upper()+','+ \
        str(pkt.pad).upper()+','+str(pkt.present).upper()+','+str(pkt.Rate).upper()+','+ \
        str(pkt.len).upper()+','+str(pkt.TXFlags).upper()+','+str(pkt.notdecoded).upper()+','+ \
        str(pkt.proto).upper()+','+str(pkt.subtype).upper()+','+str(pkt.type).upper()+','+str(pkt.cfe).upper()+','+ \
        str(pkt.FCfield).upper()+','+str(pkt.ID).upper()+','+str(pkt.SC).upper()+','+ \
        str(pkt.Flags).upper()+','+str(pkt.ChannelFrequency).upper()+','+ \
        str(pkt.dBm_AntSignal).upper()+','+str(pkt.ChannelFlags).upper()+','+ \
        str(pkt.reason).upper()+','+'HEO'+','+'human forget'
  #print(a)
  df.loc[i] = list(a.split(','))
  i=i+1 

RadioTap / 802.11 Management Disassociation 46:fb:37:XX:XX:XX (TA=SA) > 98:de:d0:XX:XX:XX (RA=DA) / Dot11Disas
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 46:fb:37:XX:XX:XX (RA=DA) / Dot11Deauth
RadioTap / 802.11 Management Deauthentification 98:de:d0:XX:XX:XX (TA=SA) > 4

In [ ]:
# Save to CSV format

df.to_csv("deauth_dissas_attacks.csv") 

In [ ]:
# Read backup CSV file 

df = pd.read_csv("./deauth_dissas_attacks.csv")
df.describe()

,Unnamed: 0,version,pad,len,proto,subtype,type,ID,SC,reason
count,1100.000000,1100.0,1100.0,1100.000000,1100.0,1100.000000,1100.0,1100.000000,1100.000000,1100.000000
mean,549.500000,0.0,0.0,17.460909,0.0,11.838182,0.0,19695.804545,8153.294545,37.438182
std,317.686953,0.0,0.0,10.442238,0.0,0.545639,0.0,18725.039006,15051.659189,72.490009
min,0.000000,0.0,0.0,8.000000,0.0,10.000000,0.0,0.000000,0.000000,0.000000
25%,274.750000,0.0,0.0,12.000000,0.0,12.000000,0.0,14849.000000,124.000000,6.000000
50%,549.500000,0.0,0.0,12.000000,0.0,12.000000,0.0,14849.000000,480.000000,7.000000
75%,824.250000,0.0,0.0,13.000000,0.0,12.000000,0.0,14849.000000,10976.000000,47.000000
max,1099.000000,0.0,0.0,36.000000,0.0,12.000000,0.0,65407.000000,60528.000000,256.000000


In [ ]:
# Drop comments/observations

df.drop(["obs"],axis=1,inplace=True)
df.head(1) 

,Unnamed: 0,time,client_addr,AP_addr,BSSID,addr4,version,pad,present,Rate,...,cfe,FCfield,ID,SC,Flags,ChannelFrequency,dBm_AntSignal,ChannelFlags,reason,label
0,0,[2021-06-02 23:56:30],FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,RATE+TXFLAGS,1.0,...,NONE,NaN,14849,0,NONE,NONE,NONE,NONE,7,skiddie


In [ ]:
# Drop index column

df = df.drop(df.columns[[0]], axis=1)
df.tail(1)

,time,client_addr,AP_addr,BSSID,addr4,version,pad,present,Rate,len,...,cfe,FCfield,ID,SC,Flags,ChannelFrequency,dBm_AntSignal,ChannelFlags,reason,label
1099,[2021-06-05 03:14:25],98:de:d0:XX:XX:XX,e4:9a:79:XX:XX:XX,98:de:d0:XX:XX:XX,NONE,0,0,TSFT+FLAGS+RATE+CHANNEL+DBM_ANTSIGNAL+RXFLAGS+...,1.0,36,...,NONE,NaN,14849,15616,FCS,2437,-38,CCK+2GHZ,8,HEO


In [ ]:
#Define and test entropy function

def Entropy(text):
    log2=lambda x:math.log(x)/math.log(2)
    exr={}
    infoc=0
    for each in text:
        try:
            exr[each]+=1
        except:
            exr[each]=1
    textlen=len(text)
    for k,v in exr.items():
        freq  =  1.0*v/textlen
        infoc+=freq*log2(freq)
    infoc*=-1
    return infoc
 
while True:
  try:
    print(Entropy(input('>>>')))
  except KeyboardInterrupt:
    print('Test over!')
    break

Test over!


In [ ]:
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [ ]:
def mapping(data,feature):
    featureMap=dict()
    count=0
    for i in sorted(data[feature].unique(),reverse=True):
        featureMap[i]=count
        count=count+1
    data[feature]=data[feature].map(featureMap)
    return data

In [ ]:
# Define in_scope_AP 
# This option is more relevant for the in-production framework since the attacks may target other neighboring AP as well 

in_scope_AP = '98:de:d0:XX:XX:XX'

In [ ]:
for index, row in df.iterrows():
  df.at[index , 'time'] = datetime.strptime(row['time'], '[%Y-%m-%d %H:%M:%S]')
  df.at[index , 'Client_BCAST'] = 1 if row['client_addr'] == 'FF:FF:FF:FF:FF:FF' else 0
  df.at[index , 'in_scope_AP'] = 1 if row['AP_addr'] == in_scope_AP or row['BSSID'] == in_scope_AP else 0
  df.at[index , 'Rate'] = 1 if row['Rate'] == '1.0' else 0
  df.at[index , 'TXFlags_NOACK'] = 1 if 'NOACK' in str(row['TXFlags']) else 0
  df.at[index , 'TXFlags_NOSEQ'] = 1 if 'NOSEQ' in str(row['TXFlags']) else 0
  df.at[index , 'TXFlags_TX_FAIL'] = 1 if 'TX_FAIL' in str(row['TXFlags']) else 0
  df.at[index , 'FCfield_RETRY'] = 1 if 'RETRY' in str(row['FCfield']) else 0
  df.at[index , 'FCS'] = 1 if 'FCS' in str(row['Flags']) else 0
  if row['dBm_AntSignal'] == 'NONE' : df.at[index , 'dBm_AntSignal'] = 0
  if row['ChannelFrequency'] == 'NONE' : df.at[index , 'ChannelFrequency'] = 0
  df.at[index , 'ChannelFlags_2GHZ'] = 1 if '2GHZ' in str(row['ChannelFlags']) else 0
  df.at[index , 'entropy_of_notdecoded'] = Entropy(row['notdecoded'])
  df.at[index , 'TSFT'] = 1 if 'TSFT' in str(row['present']) else 0
  df.at[index , 'CHANNEL'] = 1 if 'CHANNEL' in str(row['present']) else 0
  df.at[index , 'RXFLAGS'] = 1 if 'RXFLAGS' in str(row['present']) else 0
  df.at[index , 'RADIOTAPNS'] = 1 if 'RADIOTAPNS' in str(row['present']) else 0
  df.at[index , 'EXT'] = 1 if 'EXT' in str(row['present']) else 0

In [ ]:
df=df.drop(['addr4', 'version', 'pad', 'present', 'TXFlags', 'notdecoded', 'proto', 'type', 'cfe', 'FCfield', 'Flags', 'ChannelFlags'],axis=1)

df.sample(5)

,time,client_addr,AP_addr,BSSID,Rate,len,subtype,ID,SC,ChannelFrequency,...,TXFlags_TX_FAIL,FCfield_RETRY,FCS,ChannelFlags_2GHZ,entropy_of_notdecoded,TSFT,CHANNEL,RXFLAGS,RADIOTAPNS,EXT
774,2021-06-03 16:03:31,24:62:ab:XX:XX:XX,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,0,8,12,0,0,0,...,0.0,0.0,0.0,0.0,0.918296,0.0,0.0,0.0,0.0,0.0
470,2021-06-03 13:36:27,98:de:d0:XX:XX:XX,24:62:ab:XX:XX:XX,98:de:d0:XX:XX:XX,1,12,12,14849,560,0,...,0.0,0.0,0.0,0.0,0.918296,0.0,0.0,0.0,0.0,0.0
460,2021-06-03 13:36:27,24:62:ab:XX:XX:XX,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,1,12,12,14849,480,0,...,0.0,0.0,0.0,0.0,0.918296,0.0,0.0,0.0,0.0,0.0
745,2021-06-03 16:03:29,46:fb:37:XX:XX:XX,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,0,8,12,0,0,0,...,0.0,0.0,0.0,0.0,0.918296,0.0,0.0,0.0,0.0,0.0
1028,2021-06-05 03:11:35,46:fb:37:XX:XX:XX,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,1,36,12,14849,40256,2437,...,0.0,1.0,1.0,1.0,2.550341,1.0,1.0,1.0,1.0,1.0


In [ ]:
# Save preprocessed dataframe to a backup file

df.to_csv("preprocessed.csv") 

## Now that we have our primary dataset, we will like to test it with a simple deep learning algorithm

#### First, we need to import the dataset and do some cleanup

In [7]:
df = pd.read_csv("./preprocessed.csv")
df.sample(3)

,Unnamed: 0,time,client_addr,AP_addr,BSSID,Rate,len,subtype,ID,SC,ChannelFrequency,dBm_AntSignal,reason,label,same_time,same_AP_addr,same_client_addr,same_BSSID,Client_BCAST,in_scope_AP,TXFlags_NOACK,TXFlags_NOSEQ,TXFlags_TX_FAIL,FCfield_RETRY,FCS,ChannelFlags_2GHZ,entropy_of_notdecoded,TSFT,CHANNEL,RXFLAGS,RADIOTAPNS,EXT
75,75,2021-06-02 23:56:30,FF:FF:FF:FF:FF:FF,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,1,12,12,14849,608,0,0,7,skiddie,75,75,75,75,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.918296,0.0,0.0,0.0,0.0,0.0
580,580,2021-06-03 14:20:49,e4:9a:79:XX:XX:XX,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,1,36,12,14849,17776,2422,-88,1,hacker,5,19,19,19,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.732159,1.0,1.0,1.0,1.0,1.0
259,259,2021-06-03 02:45:27,f0:c3:71:XX:XX:XX,98:de:d0:XX:XX:XX,98:de:d0:XX:XX:XX,1,13,12,65407,368,0,0,7,skiddie,3,9,0,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.235926,0.0,0.0,0.0,0.0,0.0


In [8]:
df = df[[c for c in df if c not in ['label']] + ['label']]

In [9]:
df = df.drop(['time', 'client_addr', 'AP_addr', 'BSSID'], axis=1)

In [10]:
df = df.drop(df.columns[[0]], axis=1)

In [11]:
mapping = {'skiddie':0, 'HEO': 1, 'hacker':2}
df['label'] = df['label'].replace(mapping)

In [22]:
# Label encode

encoder = preprocessing.LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

y_dummy = np_utils.to_categorical(encoded_Y)

In [32]:
# Testing the resulting dataset with a simple Keras ANN implementation 

X = df.values[:,:26]

model = Sequential()
model.add(Dense(42, input_dim=26, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

# compile the Keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y_dummy, epochs=20, batch_size=3)
# evaluate the keras model
_, accuracy = model.evaluate(X, y_dummy)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/20
367/367 [==============================] - 2s 2ms/step - loss: 1252.9980 - accuracy: 0.5414
Epoch 2/20
367/367 [==============================] - 1s 2ms/step - loss: 26.7629 - accuracy: 0.7860
Epoch 3/20
367/367 [==============================] - 1s 1ms/step - loss: 25.4950 - accuracy: 0.8161
Epoch 4/20
367/367 [==============================] - 1s 1ms/step - loss: 32.5906 - accuracy: 0.8116
Epoch 5/20
367/367 [==============================] - 1s 1ms/step - loss: 30.1295 - accuracy: 0.8068
Epoch 6/20
367/367 [==============================] - 1s 1ms/step - loss: 37.4659 - accuracy: 0.8147
Epoch 7/20
367/367 [==============================] - 1s 1ms/step - loss: 22.6811 - accuracy: 0.8322
Epoch 8/20
367/367 [==============================] - 1s 1ms/step - loss: 30.6290 - accuracy: 0.8248
Epoch 9/20
367/367 [==============================] - 1s 2ms/step - loss: 19.5566 - accuracy: 0.8546
Epoch 10/20
367/367 [==============================] - 1s 1ms/step - loss: 32.3289 - accu